<a href="https://colab.research.google.com/github/gp201/BERT/blob/master/create_data_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import pandas as pd
from csv import writer

In [137]:
# tags from corpus
tagged_corpus = pd.read_csv("corpus_tags.tsv", sep="\t", names=["pmid", "start", "end", "word", "type", "umls_id"])
tagged_corpus = tagged_corpus.dropna()
tagged_corpus.head()

,pmid,start,end,word,type,umls_id
0,25763772,0,5,DCTN4,"T116,T123",C4308010
1,25763772,23,63,chronic Pseudomonas aeruginosa infection,T047,C0854135
2,25763772,67,82,cystic fibrosis,T047,C0010674
3,25763772,83,120,Pseudomonas aeruginosa (Pa) infection,T047,C0854135
4,25763772,124,139,cystic fibrosis,T047,C0010674


In [138]:
classes = []
with open("vocab_class.txt") as ftags:
    classes = [i.strip() for i in ftags.readlines()]
    ftags.close()
classes

['T047', 'T005', 'T007', 'T016']

In [0]:
def get_words_tags(text, pmid,start=0):
    words = []
    tags = []  
    row_content = []
    typ = "0"
    df = tagged_corpus[(tagged_corpus["pmid"]==int(pmid)) & (tagged_corpus["end"] <= start + len(text)) & (tagged_corpus["start"] >= start)]
    indices = df.index.values
    for word in text.split():
        for i in indices:
            text2 = df.loc[i]["word"]
            #print("#",text2.split())
            if word in text2.split() and set(df.loc[i]["type"].split(',')).intersection(classes):
                typ = df.loc[i]["type"]
                break
            else:
                typ = "0"
        row_content.append([word,typ])
    return row_content

In [0]:
title_regex = re.compile("^\d+\|t\|")
abstract_regex = re.compile("^\d+\|a\|")


fr = open("test.txt")
fw = open("labelled_data.txt", "w")

In [0]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        for row in list_of_elem:
          csv_writer.writerow(row)
        blank = ['','']
        csv_writer.writerow(blank)

In [0]:
abstract_count = 0
for i in fr.readlines():
    line = i.strip()
    title_search = title_regex.search(line)
    abstract_search = abstract_regex.search(line)
    if title_search:
        title_line = title_regex.sub('',line)
    elif abstract_search:
        abstract = abstract_regex.sub('',line)
        pmid = int(abstract_search.group(0)[:-3])
        row_check = get_words_tags(title_line,pmid,0)
        fw.write("\t".join(words)+"\n")
        fw.write("\t".join(tags)+"\n")
        title_size = len(title_line) + 1                     # 1 for \n        
        for sentence in re.split("\.", abstract):
            if len(sentence) == 0:
                continue
            _size = abstract.index(sentence) + title_size
            row_content = get_words_tags(sentence, pmid)
            append_list_as_row('data-3.csv', row_content)
        abstract_count += 1
        if abstract_count%10 == 0:
            print("PMID {} written to file. Abstract Count: {}".format(pmid, abstract_count))
fw.close()
fr.close()